In [6]:
from playwright.async_api import async_playwright, Playwright
from rich import print
import asyncio

In [ ]:
from playwright.async_api import async_playwright
from collections import defaultdict
from urllib.parse import urljoin
import nest_asyncio
import json
import re
nest_asyncio.apply()
async def run(playwright):
    start_url = "https://aniagotuje.pl"
    browser = await playwright.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto(start_url)

    recipecount=0
    recipemap = defaultdict(lambda: {"ingredient": [],"recipe_info":[]})
    while True:
        if recipecount<2:
            links = await page.locator('a[href^="/przepis/"]').all()
            if len(links)<20:
                break
            print(f"Znaleziono {len(links)} linków")
            
            linkcount=0    

            for link in links:
                if linkcount<3:
                    p = await browser.new_page(base_url=start_url)

                    href = await link.get_attribute('href')
                    
                    if "#" not in href:

                        if href not in recipemap.keys():
                            await p.goto(href)
                            print(href)
                            ingredients = await p.locator('span[class^="ingredient"]').all()
                            for ing in ingredients:
                                
                                text = await ing.text_content()
                                recipemap[href]["ingredient"].append(text)
                            recipe_info = await p.locator('p[class^="recipe-info"]').all()
                            
                            for inf in recipe_info:
                                inftext=await inf.text_content()
                                inftext=inftext.replace("\t"," ")
                                inftext=re.sub(r'\s{2,}',",",inftext)
                                inftext=inftext.split(",")
                                recipemap[href]["recipe_info"].append(str(inftext))
                else:
                    break
                linkcount=linkcount+1 
        else:
            break
        recipecount=recipecount+1
        
        next_link = page.locator('a.page-link', has_text='»')
    
        if await next_link.count() > 0:
            href = await next_link.get_attribute('href')
            full_url = urljoin(start_url, href)
            if href:
                await page.goto(full_url)
                await page.wait_for_load_state('load')
            else:
                break
        else:
            break
    await browser.close()
    return recipemap



async with async_playwright() as playwright:
    mappa = await run(playwright)

    with open("przepisy.json", "w", encoding="utf-8") as f:
        json.dump(mappa, f, ensure_ascii=False, indent=2)

Znaleziono 67 linków

/przepis/ogorki-kiszone

/przepis/ogorki-konserwowe

/przepis/jagodzianki

/przepis/placki-z-cukinii

...

/przepis/gofry

/przepis/salatka-z-ogorkow

/przepis/leczo

/przepis/ogorki-z-chili

CancelledError: 

In [35]:
print(mappa)

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished ex

defaultdict(<function run.<locals>.<lambda> at 0x00000204F6CAFC40>, {
    '/przepis/jak-zrobic-ciasto-na-nalesniki': {
        'ingredient': [
            '1 pełna szklanka i 2 łyżki mąki pszennej - 230 g',
            '3 średnie jajka - około 165 g po rozbiciu',
            '1 szklanka mleka - 250 ml',
            '1 szklanka wody - 250 ml',
            '4 łyżki oleju roślinnego - około 40 ml',
            'szczypta soli'
        ],
        'recipe_info': [
            [
                'Czas przygotowania: 5 minut\nCzas smażenia: 25 minut\nLiczba porcji: do 16 naleśników średnicy 24 
cm',
                'W 100 g:',
                'Wartość energetyczna',
                '263 kcal',
                'Węglowodany 32 g',
                '- W tym cukry 2 g',
                'Białko 9 g',
                'Tłuszcze 11 g',
                'Dieta: wegetariańska '
            ]
        ]
    },
    '/przepis/ogorki-kiszone': {
        'ingredient': [
            '2 kg małych ogórków gruntowych',
            '2 małe główki czosnku',
            'duży pęczek kopru z baldachami',
            'kawałek korzenia chrzanu oraz liście - 10 cm',
            '2 litry wrzątku',
            '4 płaskie łyżki soli kamiennej',
            'można też dać: liście chrzanu, zielone liście dębu oraz liście czarnej porzeczki, winorośli lub wiśni'
        ],
        'recipe_info': [
            [
                'Czas przygotowania: 30 minut\nCzas kiszenia ogórków: 14 dni',
                'Liczba porcji: 4 x słoik 1 litr',
                'W 100 gramach ogórków:',
                'Wartość energetyczna',
                '20 kcal',
                'Węglowodany 4 g',
                '- W tym cukry 1 g',
                'Białko 1 g',
                'Tłuszcze 0 g',
                'Dieta:',
                'bezglutenowa',
                ' wegańska',
                ' wegetariańska '
            ]
        ]
    },
    '/przepis/ogorki-konserwowe': {
        'ingredient': [
            '2 kg małych ogórków gruntowych',
            '2 szklanki octu spirytusowego 10 %',
            '2 litry wody - 8 szklanek',
            'około 3/4 szklanki cukru - 160 g',
            '2 pełne łyżki soli kamiennej - 50 g',
            '2 średnie marchewki - 140 g',
            'mała cebula - 100 g',
            '3 ząbki czosnku',
            'malutki korzeń chrzanu',
            '3 gałązki kopru z baldachami',
            '8 liści laurowych',
            '12 ziaren ziela angielskiego',
            '1 łyżeczka czarnego pieprzu w ziarnach',
            '6 płaskich łyżeczek ziaren gorczycy'
        ],
        'recipe_info': [
            [
                'Czas przygotowania: 30 minut\nCzas pasteryzacji: 6 minut\nCzas marynowania: 7 dni',
                'Liczba porcji: 6 słoików po 900 ml',
                'W 100 g:',
                'Wartość energetyczna',
                '28 kcal',
                'Węglowodany 6 g',
                '- W tym cukry 4 g',
                'Białko 1 g',
                'Tłuszcze 0 g',
                'Dieta:',
                'bezglutenowa',
                ' wegańska',
                ' wegetariańska '
            ]
        ]
    },
    '/przepis/placki-z-cukinii': {
        'ingredient': [
            '1 większa cukinia - 400 g',
            '2 małe lub średnie jajka',
            '4 łyżki mąki pszennej - 70 g',
            '1 łyżka kwaśniej śmietany 18 %',
            '1 średni ząbek czosnku',
            'garść cienkiego szczypiorku',
            'przyprawy: szczypta pieprzu i cukru oraz 1/3 łyżeczki soli',
            'oraz do smażenia: 4 łyżki oleju roślinnego'
        ],
        'recipe_info': [
            [
                'Czas przygotowania: 10 minut\nCzas smażenia placków: 30 minut\nLiczba porcji: 14 grubszych 
placuszków',
                'W 100 g:',
                'Wartość energetyczna',
                '143 kcal',
                'Węglowodany 14 g',
                '- W tym cukry 2 g',
                'Białko 6 g',
                'T